# Customer Segmentation using K-Means: Bike Sales Dataset

En este caso de estudio se muestran los pasos necesarios para identificar grupos de clientes en un conjunto de datos de ventas de bicicletas. En este caso, los clientes son tiendas de bicicletas ubicadas en distintas localidades de EEUU que compran a un distribuidor central. 

Este caso de estudio está basado en lo publicado en esta página web: https://www.business-science.io/business/2016/08/07/CustomerSegmentationPt1.html

Comenzamos importando las librerías necesarias y cargando los tres ficheros de datos de entrada: `orders.xlsx` que contiene las transacciones, `products.xlsx` que contiene el listado de bicicletas y `bikeshops.xlsx` que contiene el listado de clientes:

In [ ]:
from sklearn import preprocessing 
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import pandas as pd

orders = pd.read_excel('orders.xlsx', index_col=0, header=0)
products = pd.read_excel('bikes.xlsx', index_col=None, header=0)
customers = pd.read_excel('bikeshops.xlsx', index_col=None, header=0)

Comprobamos el contenido de cada uno de los ficheros:

In [ ]:
display(orders.head())
display(products.head())
display(customers.head())

A continuación, vamos a juntar los tres DataFrames en uno utilizando la función `pandas.merge`:

In [ ]:
ordersExtended = pd.merge(orders, customers, left_on = 'customer.id', right_on = 'bikeshop.id', how = 'left')
ordersExtended = pd.merge(ordersExtended, products, left_on = 'product.id', right_on = 'bike.id', how ='left')

In [ ]:
ordersExtended.head()

## Preparación de los datos para el agrupamiento

En este caso de estudio vamos a tratar de identificar grupos de clientes (tiendas de bicicletas) en función de los modelos de bicicleta comprados. Para ello, la unidad de medida será la cantidad de bicicletas compradas de cada modelo. El objetivo de este apartado es manipular el DataFrame de entrada (`orders.extended`) en un DataFrame que contenga tantas filas como bicicletas y tantas columnas como clientes, siendo el valor de cada celda la cantidad de unidades del modelo correspondiente compradas por la tienda.

El primer paso es crear un DataFrame llamado `customerTrends` que contenga: una fila por cada modelo de bicicleta, las correspondientes columnas que describen al modelo de bicicleta y tantas columnas como clientes hay. Para ello, se utiliza la función `groupby` para agrupar por modelo de bicicleta y luego se hace la dispersión de la columna inicial (`bikeshop.name`) utilizando la fuunción `unstack(level=0, fill_value=0)`.

In [ ]:
customerTrends = ordersExtended.groupby(
    ['bikeshop.name', 'model', 'category1','category2', 'frame', 'price']
)['quantity'].sum().unstack(level=0, fill_value=0).reset_index()

customerTrends = pd.DataFrame(customerTrends)

display(customerTrends)

A continuación, convertiremos el precio de cada modelo a una variable categórica que represente los conceptos de "precio alto" (de más de 3500) y "precio bajo" (de menos de 3500). Añadimos una columna llamada `price_range` al DataFrame `customerTrends` con esta nueva información:

In [ ]:
priceCategories = pd.DataFrame({'price_range': pd.cut(customerTrends['price'], [0, 3500, customerTrends['price'].max()])})

customerTrends = pd.concat([customerTrends, priceCategories], axis = 1).reset_index()

display(customerTrends.head())

Finalmente, debemos escalar los datos para identificar correctamente las preferencias de compra de cada cliente, pues las cantidades totales pueden presentar un problema. Por ejemplo, si comparamos al cliente A, que compra 4000 unidades del modelo 1 y 2000 del modelo 2, con el cliente B, que compra 400 unidades del modelo 1 y 40 del modelo 2, realmente el cliente B tiene preferencia por el modelo 1 pero este patrón se vería enmascarado por la gran cantidad de unidades compradas por el cliente A. Para hacer esta comparación posible, vamos a convertir las cantidades a proporciones, de manera que en cada columna (para cada cliente) tendremos el porcentaje de unidades de cada modelo sobre el total de sus compras.

In [ ]:
counts = customerTrends.loc[:, 'Albuquerque Cycles':'Wichita Speed']

proportions = counts.div(counts.sum(axis=0), axis=1)

display(proportions.head())

Y finalmente, dado que la función K-Means espera que las muestras estén en filas, debemos transponer el DataFrame:

In [ ]:
proportions = proportions.transpose()

proportions.columns = customerTrends['model']

display(proportions.head())

## Determinar el valor de k

Antes de aplicar K-Means, determinamos el mejor valor de k mediante el método de Elbow:

In [ ]:
nc = range(1,9)
kmeans = [KMeans(n_clusters=i, n_init=50) for i in nc]
scores = [kmeans[i].fit(proportions).inertia_ for i in range(len(kmeans))]

print('Scores')
display(scores)

plt.xlabel('Número de clústeres (k)')
plt.ylabel('Suma de los errores cuadráticos (inertia)')
plt.plot(nc, scores)

En base a este resultado, podemos seleccionar k = 4.

## Aplicación de K-Means

Con k = 4 aplicamos K-Means y almacenamos en un array los centroides y comprobamos que hay cuatro valores (uno por cada cluster).

In [ ]:
kmeans = KMeans(n_clusters=4).fit(proportions)
centroids = kmeans.cluster_centers_

display(len(centroids))

Le pedimos al agrupamiento obtenido que asigne una etiqueta a cada muestra del DataFrame `proportions` y utilizamos esta información para crear un DataFrame en que veamos el nombre de cada cliente y el grupo al que ha sido asignado:

In [ ]:
labels = kmeans.predict(proportions)
print(len(labels))

labels = pd.DataFrame({'cluster': labels})
customerNames = pd.DataFrame({'bikeshop': proportions.axes[0].tolist()})

customerClusters = pd.concat([customerNames, labels], axis = 1).reset_index()

customerClusters.sort_values(by=['cluster'])

## Analizar las preferencias de cada uno de los segmentos

Finalmente, concatemamos los 4 centroides al DataFrame `customerTrends` que teníamos anteriormente, creando un nuevo DataFrame llamado `customerTrends_withCentroids`:

In [ ]:
centroids_df = pd.DataFrame(centroids).transpose()
centroids_df.columns = ['0', '1', '2', '3']

customerTrends_withCentroids = pd.concat([customerTrends, centroids_df], axis = 1).reset_index()

display(customerTrends_withCentroids.head())

Podemos utilizar ahora este nuevo DataFrame para listarlo ordenado descendientemente por alguno de los nuevos centroides, de manera que podamos ver el listado de bicicletas preferente asociado a dicho grupo:

In [ ]:
customerTrends_withCentroids.sort_values(by = ['2'], ascending = False).head(n = 10)

## Ejercicios propuestos

1. En este caso de estudio, convertimos la matriz de cantidades a una matriz de proporciones para escalar los datos. Prueba a escalar la matriz de cantidades utilizando la estandarización y repite los análisis posteriores para comprobar cómo pueden variar los resultados.
2. Seleccionamos k = 4 porque al aplicar el método de Elbow nos pareció un punto a partir del cual no se ganaba mucho al añadir nuevos clusters. Sin embargo, k = 5 podría ser interesante, repite el análisis con este valor y analiza los nuevos segmentos.
3. Realiza el análisis de los Silhouette scores para este caso. En este enlace puedes encontrar información sobre cómo hacer los gráficos (https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html) con los scores, que se pueden calcular con las funciones `silhouette_score` (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html) y `silhouette_samples` (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_samples.html#sklearn.metrics.silhouette_samples).
4. El fichero `mall-customers.csv` contiene 200 transacciones con la siguiente información: género, edad, ingresos anuales y puntuación de gasto (un valor entre 1-100). Aplica K-Means tal y como hemos visto sobre este dataset.
5. Echa un vistazo al análisis realizado en el notebook `online-retail.ipynb`, un caso de estudio en el que se aplica el modelo RFM.